Getting setup
=======
You will need to [sign up for an AWS account](https://aws.amazon.com/).
Sign up is free.

Using a preloaded AMI
=====

MXNet requires several libraries.  These can be tricky to install (particularly CUDNN) so the good folks at MXNet have preload an AMI with
- C++ compiler with C++11 suports, such as gcc >= 4.8
- CUDA (CUDNN in optional) for GPU linear algebra
- BLAS (cblas, open-blas, atblas, mkl, or others) for CPU linear algebra
- opencv for image augmentations
- curl and openssl for read/write Amazon S3



This link https://console.aws.amazon.com/ec2/v2/home?region=us-east-1#LaunchInstanceWizard:ami=ami-12fd8178
will take you to the proper AMI. At the moment, this AMI is only available in the US-EAST zone but it shoudn't be too hard to copy it to others.




Request a GPU Spot Instance
=====

#### 1. Scroll down, select the "g2.2xLarge" instance and then "Next: Configure Instance Details"
For now we are going to use the smaller of the two GPU instance. But I encourge you to try out the large 8x (K40) should you feel you need.

**Pro tip:** If you are running thorugh this guide for the first time use a free or almost free micro instance. Once you gone through it without any errors, pay up for the full monty.

<img src="images/AWS_Spot_3.png" align=left>

#### 2. Request a spot intstance 
You should be on the "Step 3: Configure Instance Details" now.

We'll be using a spot instance as this is usually much cheaper. **Check the spot instance** beside purchasing options

<img src="images/AWS_Spot_4.png" align=left>

#### 3. Set your maximum price.
You'll need to place your bid for a spot instance. You can get an idea of current prices by looking at the price in the "current prices". **NOTE** That is the *current* price. You need to fill in the **Maximum Price** you are will pay. If the price of spot goes above this, your instance may be shut down. 

**g2.2xlarge:**
According to the [Spot bid advisor](https://aws.amazon.com/ec2/spot/bid-advisor/) you chances of being booted with a g2.2xlarge instance when your max price is 50% of the "On-Demand" instance is **low**. At the g2.2xlarge tier the On-Demand" instance is \\$0.70. So we'll go with a max price or **\$0.35**.

**g2.8xlarge:**
The g2.8xlarge on-demand price is currently \$2.60. There is also more demand for this instance. Entering a max price of 50% of the on-demand price puts you in the MEDIUM probabilty of being out bid. If your ok with this, I suggest **\$1.30**
<img src="images/AWS_Spot_5.png" align=left>

#### 4. Add Storage if needed

The default here is 8GBs of storage. If your AWS account is less than 1yr old, you can use up to 30 GBs for free. After that prices are still quite resonable. So how much is enough? 8 Gbs is way more than enough for Julia, MXNet and everything else you will need beside your data.

<img src="images/AWS_Spot_6.png" align=left>

#### To S3 or not to S3?

MXNet + AWS has a real nice feature. You can store you data in a persistant S3 storage and then point your instance at it. Amazon S3 is distributed data storage, which is quite convenient for host large scale datasets. However, in order to use S3 you need to set some MXNEt environment variables. [Details are here](http://mxnet.readthedocs.org/en/latest/aws.html#use-amazon-s3-to-host-data)

You have two other option. The first is just to import your data each time you start the instance. This is very simple to do with dropbox or a some other public data source. Load time are very quick and sometime the simplest solution is best. The other option is to store your data on the root drive and save it with your instance when you store it as an AMI. Either way you will need to budget addtional storage for your data. How much depends on your data. [add pre data storage size and discuss] 

#### 5: Tag Spot Request
skip past 'Step 5: Tag Spot Request' unless you need to tag. and on to the 'Configure Security Group'.

#### 6: Configure Security Group

Skip past 'Step 5: Tag Spot Request' unless you need to tag. and on to the 'Configure Security Group'.


<img src="images/AWS_Spot_7.png" align=left>

ec2-23-20-65-83.compute-1.amazonaws.com

#### Download you key

#### Waiting for your instance
Your instance will have a stauts on pending until it has been approved. At that point the status will chnage to active. This doesn't mean your instance is ready. It just means it has begun to spin up. 
Click in the intance id to go to the instance monitor.

<img src="images/AWS_Spot_8.png" align=left>

#### Find your public DNS
Your instance will have a stauts on pending until it has been approved. At that point the status will chnage to active. This doesn't mean your instance is ready. It just means it has begun to spin up. 
click in the intance idea to go to the instance monitor. 

<img src="images/AWS_Spot_9.png" align=left>

Setting up your instance
====
Open a terminal and switch to the folder where you stored your AWS key


    ...
     cd /Users/andrep/Documents/AWS_Key
     chmod 400 MX_Spot.pem
     ssh -v -i MX_Spot.pem ubuntu@ec2-23-20-207-135.compute-1.amazonaws.com
    ... 
    
    
You should see output ending in 

    ...
     The authenticity of host 'ec2-23-20-207-135.compute-1.amazonaws.com (23.20.207.135)' can't be established.
     RSA key fingerprint is 87:92:bf:57:c1:a1:84:74:06:27:b5:64:a1:04:b2:ae.
     Are you sure you want to continue connecting (yes/no)?
    ... 

Type '**yes**' and enter. You should see a confirmation, and lot of information and finally your prompt should change to your new AWS instance.
    ...

    ubuntu@ip-10-47-202-184:~$  
    

*note: your ip will be different from mine*

##### if you got this far, congradulations! You have a working AWS GPU instance!    

Setting up MXNet
===

[The official MXNet AWS document is here](http://mxnet.readthedocs.org/en/latest/aws.html)

Here is the condensed version. Run the following two commands.

    sudo apt-get update
    sudo apt-get install -y build-essential git libcurl4-openssl-dev libatlas-base-dev libopencv-dev python-numpy
    
This will install all the tools and dependancies needed to build MXNet. Running these two should only take about 1-2 min.

##### Now you are ready to install MXNet

Run the following to install and make MXNet. 

    git clone --recursive https://github.com/dmlc/mxnet
    cd mxnet; cp make/config.mk .
    echo "USE_CUDA=1" >>config.mk
    echo "USE_CUDA_PATH=/usr/local/cuda" >>config.mk
    echo "USE_CUDNN=1" >>config.mk
    echo "USE_BLAS=atlas" >> config.mk
    echo "USE_DIST_KVSTORE = 1" >>config.mk
    

If you intend to use S3 add this line 

    echo "USE_S3=1" >>config.mk
    
Finally, build MXNet...

    make -j8

The whole installation takes about 2 minutes. Congradulations. You have installed MXNet.
if you have setup S3 you can run a python script (we still need to install Julia). This will fail if S3 isn't setup.

    python tests/python/gpu/test_conv.py


Installing Jupyter
===

Install Jupyter seperately and before installing IJulia will make the custom configurations easy. IJulia will automatically detect and use the installed Jupyter including the custom config. We're only need the minimal 'miniconda' installation. Use the following commands to download miniconda and install.

    wget https://repo.continuum.io/miniconda/Miniconda-latest-Linux-x86_64.sh
    bash Miniconda-latest-Linux-x86_64.sh
    
You will be prompted with a license message. Click through that to until you get to the "Do you agree" and type 'yes'. You will also be prompted for a location. Just hit enter and select the default locations. Final, you'll be asked whether 'you wish the installer to prepend the Miniconda2 install location?" Select 'yes'.    

Once you're done with this, you will need to log out once and then log back in to have your conda commands recognized. Type ctrl+d to log out.  Then use the same log in command as above to log back in.

    ssh -i MX_Spot.pem ubuntu@[your public DNS]
    
Now we're ready to install Jupyter. Using the following command.

    conda install jupyter
    
Type 'yes' when prompted about the installations. The whole installation will take about 2 min.

Configuring AWS <-> Jupyter remote access
===

There are three steps to this. 

1. Setup an openssl certificate
2. Generate a sha password
3. Generate and customize a Jupyter config file

### 1. Setup a certificate

From your prompt create a new directory and change to it.
    
    mkdir ~/certificates
    cd ~/certificates
    
Now create a certificate

    openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem
    
    cd ..

You'll be prompted for 7 items of information. Just hit enter to select the defaults. 

### 2. SHA password

Start up IPython and use it to generate a SHA password.

    ipython

From within IPython run these two lines. 

    from IPython.lib import passwd
    passwd()
    
You will be prompted for a password. I'm going to chose '**JuilaMxnetAws**'. This will generate a SHA password. Mine was...

    'sha1:42c2278aaacf:33ed67eaa908630876bb7e80ea68eb855a3c5d73'
    
Take note of your SHA password and the one you entered. You will need them for the config and remote log in. Leave  ipython by typing...

    exit()

### 3. Create a config file

To create a default config file and begin editing it enter the following

    jupyter notebook --generate-config
    cd .jupyter/ 
    vim jupyter_notebook_config.py

This will put you into the vim text editor. Now, I have to confess I'm not a vim guy. If there is a better way to do this please let me know.

1. Use **Shift + G** to move to the end of the file.
2. Type 'o' to enter insert mode and enter the following text. Be sure to replace the SHA with the passowrd you generated above.

        c = get_config()

        # Kernel config
        c.IPKernelApp.pylab = 'inline'  # if you want plotting support always

        # Notebook config
        c.NotebookApp.certfile = u'/home/ubuntu/certificates/mycert.pem'
        c.NotebookApp.ip = '*'
        c.NotebookApp.open_browser = False
        c.NotebookApp.password = u'sha1:42c2278aaacf:33ed67eaa908630876bb7e80ea68eb855a3c5d73'

        # It is a good idea to put it on a known, fixed port
        c.NotebookApp.port = 8998

3. Save and exit your file by typing **ESC**, then '**:wq**'  
